## Benefits of preserving text structure:

- Better chunking that respects semantic boundaries
- More accurate search and retrieval
- Improved question answering by maintaining context
- The ability to handle structured data like tables


In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import MarkdownNodeParser
import textwrap

# Sample markdown document with clear structure
markdown_text = """
# AI Engineering Fundamentals

## Introduction to Vector Databases

Vector databases are specialized database systems designed to store and query vector embeddings efficiently.

### Key Advantages
- Efficient similarity search
- Scalable to billions of vectors
- Support for metadata filtering

### Common Operations
1. **Vector Indexing**: Creating data structures for efficient search
2. **Approximate Nearest Neighbor Search**: Finding similar vectors quickly
3. **Hybrid Search**: Combining vector similarity with metadata filters

## Working with Embeddings

Embeddings are dense numerical representations of data that capture semantic meaning.

### Popular Embedding Models
- OpenAI text-embedding-ada-002
- Sentence Transformers
- CLIP for image embeddings
"""

# Create a document
document = Document(text=markdown_text)

# Create a parser that recognizes markdown structure
markdown_parser = MarkdownNodeParser()

# Parse the document
nodes = markdown_parser.get_nodes_from_documents([document])

# Display the resulting nodes
print(f"Total nodes created: {len(nodes)}")
for i, node in enumerate(nodes):
    print(f"\nNode {i+1}:")
    print(f"Text: {textwrap.shorten(node.text, width=60)}...")
    print(f"Metadata: {node.metadata}")